In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Importing pandas & numpy library
2. Importing Seaborn & Matplotlib to image rendering
3. Importing IMread from Open CV
4. Text library to make longer to multiple lines
5. Library to read images without completly loading file
6. Tensor flow library required for Text + image processing + model
7. NLTK library for text processing + other text library

In [ ]:
import pandas as  pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
from cv2 import imread

In [ ]:
import textwrap

In [ ]:
pip install imagesize

In [ ]:

import imagesize

In [ ]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
# NLTK library for text processing
import nltk
from string import punctuation

In [ ]:
# Sklearn library for textprocessing & clustering

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

# Required Data path
#     1. train_csv --> Info about Train images + caption info 
#     2. test_csv  --> Has Test images + caption info 
#     3. train_dir --> Directory containing train images
#     4. test_dir  --> Directory containing test images
#     5. train_df  --> train_csv file read to pandas data frame for further processing 
#     6. test_df   --> test_csv file read to pandas data frame for further processing

In [ ]:
# all path info 
train_csv = "../input/shopee-product-matching/train.csv"
test_csv = "../input/shopee-product-matching/test.csv"
train_dir = "../input/shopee-product-matching/train_images/"
test_dir ="../input/shopee-product-matching/test_images/"

In [ ]:
# reading train_csv & test_cs to data frame

train_df = pd.read_csv( train_csv, sep =",")
test_df  = pd.read_csv( test_csv, sep =",")
print ("Current dimension of train data frame ={}".format( train_df.shape))
print ("Current dimension of test data frame ={}".format( test_df.shape))

# Checking train_df ( train data frame ) to check how data table is.

In [ ]:
train_df.head(10)

# Lets chek for number of categories present in the data frame train_df

In [ ]:
print ( "Number of categories present in the train df ={}".format( train_df["label_group"].nunique()) )

# 11014 unique groups are present in the data frame

# Lets check number of unique image present in the train_df 

In [ ]:
print ( "Number of Unique images present in the train df ={}".format( train_df["image"].nunique()) )

# We have 32412 unique images, were as total number of rows we have is 34250
# Roughly we mush have good number sample images for each group

In [ ]:
print ("Max number of images present for each group/category ={}".format( train_df.groupby( 'label_group').count().max().iloc[0] ) )
print ("Min number of images present for each group/category ={}".format( train_df.groupby( 'label_group').count().min().iloc[0] ) )

# Lets visualize few images with, caption with 3X3 grid
    * We have to perform 2 operation 
        1. to read image to numpy array 
        2. then plot the read data to image along with Text.

In [ ]:
train_df[["image","label_group","title"]].iloc[23].values

In [ ]:
def create_image():
    fig = plt.figure( figsize = (7,7), dpi = 80 )
    #for i in range( 0, 4 ):
    plt.subplot ( 1, 2, 1 )    
    random_num = np.random.randint(train_df.shape[0])
    image,group,caption = train_df[["image","label_group","title"]].iloc[random_num].values

    plt.imshow( imread(train_dir + image ))
    plt.title( "group = " +str(group))
    plt.subplot ( 1, 2,2 )
    plt.text( 0.5,0.5 , "\n".join( textwrap.wrap( caption, 20 ) ) )



In [ ]:
create_image()
create_image()

    # Not all words in the caption are english, have take care of it as well 

# Lets check dimension of all images, how see the dimension spread

In [ ]:
dimension = np.zeros( shape = ( train_df["image"].nunique(),2)  )

In [ ]:
for i, x in enumerate( train_df["image"].unique() ) :
    dimension[i] = imagesize.get ( train_dir + x )

In [ ]:
fig = plt.figure( figsize = (5,5), dpi =90 )
sns.scatterplot( x = [ x[0] for x  in dimension ], y = [ x[1] for x  in dimension ] )


# Dimension of image is not uniform, wehave scale image toresonable size

# Text Processing usig NLTK

    1. Tokenizing Text data
    2. Remove stop words 
    

In [ ]:
def remove_stop_words_and_tokenize( data_list ):

    processed_text_data = []
    stop_words = nltk.corpus.stopwords.words ("english")  +list(punctuation)

    for x in data_list:

        nltk_tokenized_data = nltk.word_tokenize( x.lower()  )
        nltk_tokenized_data = [ x for x in nltk_tokenized_data if  ( (x not in stop_words) and x.isalpha() and len(x) >1 ) ]
        #nltk_tokenized_data = [ x for x in nltk_tokenized_data if len(x) >1 ]
        processed_text_data.append( " ".join ( nltk_tokenized_data) )

    print ("length of tokenized words from train data = {}".format(len(processed_text_data) ) )
    print ( "first 10 words = {}".format( processed_text_data[3:10]))
    
    return ( processed_text_data )

    * Adding new text to train_df pandas data frame

In [ ]:
train_df["new_title"] = remove_stop_words_and_tokenize( train_df["title"])
train_df.head()

# >Lets check captions and its features

In [ ]:
#Tokanizing text to words & removingspecial characters 

tokenized_data = keras.preprocessing.text.Tokenizer ( num_words = None,
                                                    filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',
                                                    lower = True,
                                                    oov_token ="<unk>",
                                                    split=" ")
tokenized_data.fit_on_texts ( train_df["new_title"] )
tokenized_data.word_index["<pad>"] = 0
tokenized_data.index_word[0] ="<pad>"
train_seq_text = tokenized_data.texts_to_sequences ( train_df["new_title"])

In [ ]:
print ("total number of words are ={}".format ( len ( tokenized_data.word_counts)  ) )

In [ ]:

words_df = pd.DataFrame( data ={ "words":[ x for x,y in tokenized_data.word_counts.items()],
                                "count":[ y for x,y in tokenized_data.word_counts.items()]}
                       )
words_df.sort_values( "count", inplace = True,ascending = False )
words_df.head()

In [ ]:
fig ,axis = plt.subplots( nrows = 2, ncols = 1,  figsize =(20,20 ), dpi = 90 )
sns.barplot( data =words_df.head(100), x = "words",y ="count", ax = axis [0])
axis[0].tick_params(axis="x", rotation = 90 )
axis[0].title.set_text( " First 100 most common words ")
sns.barplot( data =words_df.tail(100), x = "words",y ="count", ax = axis [1])
axis[1].tick_params(axis="x", rotation = 90 )
axis[1].title.set_text( " last 100 most common words ")

# As we have close 18k words, Picking 10K frequent words for model building 

In [ ]:
#Tokanizing text to words & removingspecial characters 

tokenized_data = keras.preprocessing.text.Tokenizer ( num_words = None,
                                                    filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n',
                                                    lower = True,
                                                    oov_token ="<unk>",
                                                    split=" ")
tokenized_data.fit_on_texts ( train_df["new_title"] )
tokenized_data.word_index["<pad>"] = 0
tokenized_data.index_word[0] ="<pad>"
train_seq_text = tokenized_data.texts_to_sequences ( train_df["new_title"])
print ("total number of words are ={}".format ( len ( tokenized_data.word_counts)  ) )

In [ ]:
train_df["caption_vector"] = train_seq_text
train_df.head()

In [ ]:
NGRAMS_TXT = 3
def remove_stop_words_and_tokenize_tf_idf( data_list, NGRAMS_TXT = NGRAMS_TXT ):

    #processed_text_data = []
    stop_words = nltk.corpus.stopwords.words ("english")  +list(punctuation)
    #print (data_list)
    #for x in data_list:

    nltk_tokenized_data = nltk.word_tokenize( data_list.lower()  )
    nltk_tokenized_data = [ x for x in nltk_tokenized_data if  ( (x not in stop_words) and x.isalpha() and len(x) >1 ) ]
    nltk_tokenized_data = [ x for x in nltk_tokenized_data if len(x) >1 ]
    #n_grams_data= list ( nltk.ngrams( nltk_tokenized_data, NGRAMS_TXT ) )
    #processed_text_data.append( " ".join ( nltk_tokenized_data) )

    #print ("length of tokenized words from train data = {}".format(len(processed_text_data) ) )
    #print ( "first 10 words = {}".format( processed_text_data[3:10]))
    
    return ( nltk_tokenized_data )#+ n_grams_data )

In [ ]:
processed_data = remove_stop_words_and_tokenize_tf_idf (train_df["title"].iloc[4] )
print( "processed data = {}".format(processed_data))
print (  train_df["title"].iloc[10] )

In [ ]:
#NGRAMS_TXT = 
tf_idf = TfidfVectorizer( analyzer= remove_stop_words_and_tokenize_tf_idf,ngram_range= (1,4)).fit( train_df["title"] )
#tf_idf = TfidfVectorizer( ngram_range= (1,2),lowercase=True,stop_words= "english")
#tf_idf.fit( train_df["title"] )

In [ ]:
print ("Lenght of vocabulary ={} words".format( len ( tf_idf.vocabulary_) ) )

In [ ]:
transformed_tf_idf = tf_idf.fit_transform( train_df["title"])

In [ ]:
 np.where ( cosine_similarity( transformed_tf_idf[6], transformed_tf_idf[1:] ) > 0.8)[1] 

In [ ]:
label= train_df["label_group"].iloc[4]
group = train_df.groupby( by ="label_group" )
group.get_group( label)

In [ ]:
transformed_tf_idf[4] 

In [ ]:
group.get_group( label)

In [ ]:
plt.imshow( imread(r"../input/shopee-product-matching/train_images/" + "00117e4fc239b1b641ff08340b429633.jpg" ))

In [ ]:
plt.imshow( imread(r"../input/shopee-product-matching/train_images/" + "52f5b2e6f6647325817eb99db17709f0.jpg" ))

In [ ]:
## Tensorflow Extracting features from iamge 

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
img, path = load_image( "../input/shopee-product-matching/train_images/" + train_df["image"].iloc[1] )

In [ ]:
cosine_similarity ( img,img)